In [135]:
import numpy as np
import pandas as pd
import psycopg2
%matplotlib inline
import datetime

In [136]:
# create a database connection
sqluser = 'ephwha'
dbname = 'mimic'
schema_name = 'mimic'

In [137]:
pd.set_option('display.max_rows', None)
con = psycopg2.connect(dbname=dbname, user=sqluser, password='mimic')
cur = con.cursor()

In [193]:
query = """
with raw as (
select icustays.icustay_id
, icustays.intime
, icustays.intime + interval '24' hour as in24
, icustays.intime + interval '48' hour as in48
, icustays.intime + interval '72' hour as in72
, vd.starttime, vd.endtime
from icustays
join vasopressordurations vd
on icustays.icustay_id = vd.icustay_id
),
single_durations as (
select *,
case 
    when in24 > starttime 
    then EXTRACT(epoch FROM in24 - starttime - GREATEST(interval '0' hour, in24 - endtime)) / 3600 
    else 0 
end as dur24,
case 
    when in48 > starttime 
    then EXTRACT(epoch FROM in48 - starttime - GREATEST(interval '0' hour, in48 - endtime)) / 3600 
    else 0 
end as dur48,
case 
    when in72 > starttime 
    then EXTRACT(epoch FROM in72 - starttime - GREATEST(interval '0' hour, in72 - endtime)) / 3600 
    else 0 
end as dur72
from raw
)
select icustay_id
, sum(dur24) as first_24_vasso_hours
, sum(case when dur24 > 0 then 1 else 0 end) as first_24_vasso_counts
, sum(dur48) as first_48_vasso_hours
, sum(case when dur48 > 0 then 1 else 0 end) as first_48_vasso_counts
, sum(dur72) as first_72_vasso_hours
, sum(case when dur72 > 0 then 1 else 0 end) as first_72_vasso_counts
from single_durations
group by icustay_id
order by icustay_id
"""

df = pd.read_sql(query, con)
df.head()

,icustay_id,first_24_vasso_hours,first_24_vasso_counts,first_48_vasso_hours,first_48_vasso_counts,first_72_vasso_hours,first_72_vasso_counts
0,200003,1.084444,1,17.334444,3,31.500000,3
1,200024,4.333333,1,4.333333,1,4.333333,1
2,200025,4.750000,1,4.750000,1,4.750000,1
3,200028,20.733333,1,20.733333,1,20.733333,1
4,200029,1.500000,1,1.500000,1,1.500000,1


In [194]:
data.to_csv('vasso_durations.csv', index=False)

### Table 1

In [132]:
query = """
-- prep height
WITH ht_stg0 AS
(
  SELECT 
    c.subject_id, c.icustay_id, c.charttime,
    -- Ensure that all heights are in centimeters
    CASE
      WHEN c.itemid IN (920, 1394, 4187, 3486, 226707)
        THEN c.valuenum * 2.54
      ELSE c.valuenum
    END AS height
  FROM chartevents  c
  WHERE c.valuenum IS NOT NULL
  AND c.valuenum != 0
  -- exclude rows marked as error
  AND COALESCE(c.error, 0) = 0
  AND c.itemid IN
  (
    -- CareVue
    920, 1394, 4187, 3486,                    -- Height inches
    3485, 4188                                -- Height cm
    -- Metavision
    , 226707 -- Height (measured in inches)
    -- note we intentionally ignore the below ITEMID in metavision
    -- these are duplicate data in a different unit
    -- , 226730 -- Height (cm)
  )
)
-- filter out bad heights
, ht_stg1 AS
(
  SELECT
    icustay_id
    , charttime
    , CASE
        -- rule for neonates
        WHEN charttime <= (pt.dob + INTERVAL '1' year) AND height < 80 THEN height
        -- rule for adults
        WHEN charttime >  (pt.dob + INTERVAL '1' year) AND height > 120 AND height < 230 THEN height
      ELSE NULL END as height
  FROM ht_stg0 h
  INNER JOIN patients  pt
    ON h.subject_id = pt.subject_id
), hw as (
SELECT 
  ie.icustay_id,
  ROUND(CAST(wt.weight_first AS NUMERIC), 2) AS weight_first,
  ROUND(CAST(wt.weight_last AS NUMERIC), 2) AS weight_last,
  ROUND(CAST(wt.weight_min AS NUMERIC), 2) AS weight_min,
  ROUND(CAST(wt.weight_max AS NUMERIC), 2) AS weight_max,
  ROUND(CAST(ht.height_first AS NUMERIC), 2) AS height_first,
  ROUND(CAST(ht.height_last AS NUMERIC), 2) AS height_last,
  ROUND(CAST(ht.height_min AS NUMERIC), 2) AS height_min,
  ROUND(CAST(ht.height_max AS NUMERIC), 2) AS height_max
FROM icustays ie
-- get weight from weightdurations table
LEFT JOIN
(
  SELECT icustay_id,
    MIN(CASE WHEN rn_asc = 1 THEN weight ELSE NULL END) as weight_first,
    MIN(CASE WHEN rn_desc = 1 THEN weight ELSE NULL END) as weight_last,
    MIN(weight) AS weight_min,
    MAX(weight) AS weight_max
  FROM
  (
    SELECT
      icustay_id,
      weight,
      ROW_NUMBER() OVER (PARTITION BY icustay_id ORDER BY starttime asc) as rn_asc,
      ROW_NUMBER() OVER (PARTITION BY icustay_id ORDER BY starttime desc) as rn_desc
    FROM weightdurations
  ) wt_stg
  GROUP BY icustay_id
) wt
  ON ie.icustay_id = wt.icustay_id
-- get first/min/max height from above, after filtering bad data
LEFT JOIN
(
  SELECT icustay_id,
    MIN(CASE WHEN rn_asc = 1 THEN height ELSE NULL END) as height_first,
    MIN(CASE WHEN rn_desc = 1 THEN height ELSE NULL END) as height_last,
    MIN(height) AS height_min,
    MAX(height) AS height_max
  FROM
  (
    SELECT
      icustay_id,
      height,
      ROW_NUMBER() OVER (PARTITION BY icustay_id ORDER BY charttime asc) as rn_asc,
      ROW_NUMBER() OVER (PARTITION BY icustay_id ORDER BY charttime desc) as rn_desc
    FROM ht_stg1
  ) ht_stg2
  GROUP BY icustay_id
) ht
  ON ie.icustay_id = ht.icustay_id
ORDER BY icustay_id)
, vtd as (
select 
icustay_id, sum(duration_hours) / 24.0 as vent_duration_days
from ventdurations
where icustay_id is not null
group by icustay_id 
)
--- ,raw_fluid as (
--- select *, datetime_diff(endtime, starttime, MINUTE) as dur, datetime_diff(endtime, starttime, MINUTE) / 60.0 * rate_all as amt
--- from fluid_balance
--- where icustay_id is not null)
--- ,fluid as (
--- select icustay_id, sum(amt) as nettotal, max(endtime) as last_fluid_time 
--- from raw_fluid
--- group by ICUSTAY_ID)
, add_cormor as (
select hadm_id, 
max(case when icd9_code like '4273%' then 1 else 0 end) as afib,
max(case when icd9_code in  ('5854','5855','5856') then 1 else 0 end) as ckd,
max(case when icd9_code like '571%' then 1 else 0 end) as cld,
max(case when icd9_code in
  (  '4660 ','490  ','4910 ','4911 ','49120'
    ,'49121','4918 ','4919 ','4920 ','4928 '
    ,'494  ','4940 ','4941 ','496  ') then 1 else 0 end) as copd,
max(case when icd9_code similar to '(433|434|435|436|437)%' then 1 else 0 end) as cereb,
max(case when icd9_code similar to '(410|411|412|413)%' then 1 else 0 end) as ihd,
max(case when SUBSTR(icd9_code, 1, 3) in
  (
     '140','141','142','143','144','145','146','147','148','149','150','151','152'
    ,'153','154','155','156','157','158','159','160','161','162','163','164','165'
    ,'166','167','168','169','170','171','172','174','175','176','177','178','179'
    ,'180','181','182','183','184','185','186','187','188','189','190','191','192'
    ,'193','194','195','196','197','198','199','200','201','202','203','204','205'
    ,'206','207','208','209','230','231','232','233','234','235','236','237','238','239'
  ) then 1 else 0 end) as cancer
from diagnoses_icd
group by hadm_id
)
, fluid_cv_0 as (
    select icustay_id, itemid, amount, amountuom, charttime
    from inputevents_cv
    where itemid in (30058,30065,
        30013,
        30187,30016,30317,30318,
        30018,30296,30190,
        30021,
        30020,30160,30159,
        30143,30061,
        30015,30060,
        30030,
        30008,
        30032,30196,30096,30197,30198,30199,30200,30325,30194,30313,30301,30191,30193,30192,30323,30314,30203,
        30099,
        30208)
    and amountuom ~* 'ml'
    and amount > 0
)

, fluid_mv_0 as (
    select icustay_id, itemid,
        case when amountuom similar to '(l|L)' then amount * 1000 else amount end as amount,
        amountuom, starttime, endtime
    from inputevents_mv
    where itemid in (225944,
        220949,
        220952,220950,
        225158,
        225943,226089,
        225828,227533,
        225159,
        225823,225825,225827,225941,225823,
        225161,
        220995,
        220862,220864,
        225916,225917,225948,225947,
        225920,
        225969)
    and amountuom ~* 'ml|l'
    and starttime <= endtime
    and amount > 0
)

, fluid_mv_3 as (
    select icustay_id, sum(amount) as amount
    from fluid_mv_0
    group by icustay_id
)

, fluid_cv_3 as (
    select icustay_id, sum(amount) as amount
    from fluid_cv_0
    group by icustay_id
)

, fluid_0 as (
    select * from fluid_mv_3
    union
    select * from fluid_cv_3
)

, fluid_1 as (
    select icustay_id, sum(amount) as total_input
    from fluid_0
    group by icustay_id
)
, uo as (
    select icustay_id, sum(value) as total_output
    from urineoutput 
    group by icustay_id
)
, fluid_balance as (
select fluid_1.*, uo.total_output, total_input-total_output as nettotal from fluid_1
left join uo using (icustay_id)
),

------------

cr as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , le.valuenum as creat
  , le.charttime
  from icustays ie
  left join labevents le
    on ie.subject_id = le.subject_id
    and le.ITEMID = 50912
    and le.VALUENUM is not null
    and le.CHARTTIME between (ie.intime - interval '7' day) and (ie.intime + interval '3' day)
)
, kdigo_creat as (
-- add in the lowest value in the previous 48 hours/7 days
SELECT
  cr.icustay_id
  , cr.charttime
  , cr.creat
  , MIN(cr24.creat) AS creat_low_past_24hr
  , MIN(cr48.creat) AS creat_low_past_48hr
  , MIN(cr72.creat) AS creat_low_past_72hr
  , MIN(cr72.creat) AS creat_low_past_7day
FROM cr
LEFT JOIN cr cr24
  ON cr.icustay_id = cr24.icustay_id
  AND cr24.charttime <  cr.charttime
  AND cr24.charttime >= (cr.charttime - INTERVAL '24' HOUR)
LEFT JOIN cr cr48
  ON cr.icustay_id = cr48.icustay_id
  AND cr48.charttime <  cr.charttime
  AND cr48.charttime >= (cr.charttime - INTERVAL '48' HOUR)
LEFT JOIN cr cr72
  ON cr.icustay_id = cr72.icustay_id
  AND cr72.charttime <  cr.charttime
  AND cr72.charttime >= (cr.charttime - INTERVAL '72' HOUR)
LEFT JOIN cr cr7d
  ON cr.icustay_id = cr7d.icustay_id
  AND cr7d.charttime <  cr.charttime
  AND cr7d.charttime >= (cr.charttime - INTERVAL '7' DAY)
GROUP BY cr.icustay_id, cr.charttime, cr.creat
ORDER BY cr.icustay_id, cr.charttime, cr.creat)

-----
, ur_stg as
(
  select io.icustay_id, io.charttime
  -- we have joined each row to all rows preceding within 24 hours
  -- we can now sum these rows to get total UO over the last 24 hours
  -- we can use case statements to restrict it to only the last 6/12 hours
  -- therefore we have three sums:
  -- 1) over a 6 hour period
  -- 2) over a 12 hour period
  -- 3) over a 24 hour period
  -- note that we assume data charted at charttime corresponds to 1 hour of UO
  -- therefore we use '5' and '11' to restrict the period, rather than 6/12
  -- this assumption may overestimate UO rate when documentation is done less than hourly

  -- 6 hours
  , sum(case when io.charttime <= iosum.charttime + interval '5' hour
      then iosum.VALUE
    else null end) as UrineOutput_6hr
  -- 12 hours
  , sum(case when io.charttime <= iosum.charttime + interval '11' hour
      then iosum.VALUE
    else null end) as UrineOutput_12hr
  -- 24 hours
  , sum(iosum.VALUE) as UrineOutput_24hr
  -- calculate the number of hours over which we've tabulated UO
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - 
        -- below MIN() gets the earliest time that was used in the summation 
        MIN(case when io.charttime <= iosum.charttime + interval '5' hour
          then iosum.charttime
        else null end)
    -- convert from EPOCH (seconds) to hours by dividing by 360.0
    )/3600.0 AS NUMERIC), 4) AS uo_tm_6hr
  -- repeat extraction for 12 hours and 24 hours
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - 
        MIN(case when io.charttime <= iosum.charttime + interval '11' hour
          then iosum.charttime
        else null end)
   )/3600.0 AS NUMERIC), 4) AS uo_tm_12hr
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - MIN(iosum.charttime)
   )/3600.0 AS NUMERIC), 4) AS uo_tm_24hr
  from urineoutput io
  -- this join gives all UO measurements over the 24 hours preceding this row
  left join urineoutput iosum
    on  io.icustay_id = iosum.icustay_id
    and io.charttime >= iosum.charttime
    and io.charttime <= (iosum.charttime + interval '23' hour)
  group by io.icustay_id, io.charttime
)
, kdigo_uo as (
select
  ur.icustay_id
, ur.charttime
, wd.weight
, ur.UrineOutput_6hr
, ur.UrineOutput_12hr
, ur.UrineOutput_24hr
-- calculate rates - adding 1 hour as we assume data charted at 10:00 corresponds to previous hour
, ROUND((ur.UrineOutput_6hr/wd.weight/(uo_tm_6hr+1))::NUMERIC, 4) AS uo_rt_6hr
, ROUND((ur.UrineOutput_12hr/wd.weight/(uo_tm_12hr+1))::NUMERIC, 4) AS uo_rt_12hr
, ROUND((ur.UrineOutput_24hr/wd.weight/(uo_tm_24hr+1))::NUMERIC, 4) AS uo_rt_24hr
-- time of earliest UO measurement that was used to calculate the rate
, uo_tm_6hr
, uo_tm_12hr
, uo_tm_24hr
from ur_stg ur
left join weightdurations wd
  on  ur.icustay_id = wd.icustay_id
  and ur.charttime >= wd.starttime
  and ur.charttime <  wd.endtime
order by icustay_id, charttime
)

-----

, cr_stg AS
(
  SELECT
    cr.icustay_id
    , cr.charttime
    , cr.creat
    , case
        -- 3x baseline
        when cr.creat >= (cr.creat_low_past_7day*3.0) then 3
        -- *OR* cr >= 4.0 with associated increase
        when cr.creat >= 4
        -- For patients reaching Stage 3 by SCr >4.0 mg/dl
        -- require that the patient first achieve ... acute increase >= 0.3 within 48 hr
        -- *or* an increase of >= 1.5 times baseline
        and (cr.creat_low_past_48hr <= 3.7 OR cr.creat >= (1.5*cr.creat_low_past_7day))
            then 3 
        -- TODO: initiation of RRT
        when cr.creat >= (cr.creat_low_past_7day*2.0) then 2
        when cr.creat >= (cr.creat_low_past_48hr+0.3) then 1
        when cr.creat >= (cr.creat_low_past_7day*1.5) then 1
    else 0 end as aki_stage_creat
  FROM kdigo_creat cr
)
-- stages for UO / creat
, uo_stg as
(
  select
      uo.icustay_id
    , uo.charttime
    , uo.weight
    , uo.uo_rt_6hr
    , uo.uo_rt_12hr
    , uo.uo_rt_24hr
    -- AKI stages according to urine output
    , CASE
        WHEN uo.uo_rt_6hr IS NULL THEN NULL
        -- require patient to be in ICU for at least 6 hours to stage UO
        WHEN uo.charttime <= ie.intime + interval '6' hour THEN 0
        -- require the UO rate to be calculated over half the period
        -- i.e. for uo rate over 24 hours, require documentation at least 12 hr apart
        WHEN uo.uo_tm_24hr >= 11 AND uo.uo_rt_24hr < 0.3 THEN 3
        WHEN uo.uo_tm_12hr >= 5 AND uo.uo_rt_12hr = 0 THEN 3
        WHEN uo.uo_tm_12hr >= 5 AND uo.uo_rt_12hr < 0.5 THEN 2
        WHEN uo.uo_tm_6hr >= 2 AND uo.uo_rt_6hr  < 0.5 THEN 1
    ELSE 0 END AS aki_stage_uo
  from kdigo_uo uo
  INNER JOIN icustays ie
    ON uo.icustay_id = ie.icustay_id
)
-- get all charttimes documented
, tm_stg AS
(
    SELECT
      icustay_id, charttime
    FROM cr_stg
    UNION
    SELECT
      icustay_id, charttime
    FROM uo_stg
)
, kdigo_stages as (select
    ie.icustay_id
  , tm.charttime
  , cr.creat
  , cr.aki_stage_creat
  , uo.uo_rt_6hr
  , uo.uo_rt_12hr
  , uo.uo_rt_24hr
  , uo.aki_stage_uo
  -- Classify AKI using both creatinine/urine output criteria
  , GREATEST(cr.aki_stage_creat, uo.aki_stage_uo) AS aki_stage
FROM icustays ie
-- get all possible charttimes as listed in tm_stg
LEFT JOIN tm_stg tm
  ON ie.icustay_id = tm.icustay_id
LEFT JOIN cr_stg cr
  ON ie.icustay_id = cr.icustay_id
  AND tm.charttime = cr.charttime
LEFT JOIN uo_stg uo
  ON ie.icustay_id = uo.icustay_id
  AND tm.charttime = uo.charttime
order by ie.icustay_id, tm.charttime)

, cr_aki AS
(
  SELECT
    k.icustay_id
    , k.charttime
    , k.creat
    , k.aki_stage_creat
    , ROW_NUMBER() OVER (PARTITION BY k.icustay_id ORDER BY k.aki_stage_creat DESC, k.creat DESC) AS rn
  FROM icustays ie
  INNER JOIN kdigo_stages k
    ON ie.icustay_id = k.icustay_id
  WHERE k.charttime > (ie.intime - interval '6' hour)
  AND k.charttime <= (ie.intime + interval '48' hour)
  AND k.aki_stage_creat IS NOT NULL
)
-- get the worst staging of urine output in the first 48 hours
, uo_aki AS
(
  SELECT
    k.icustay_id
    , k.charttime
    , k.uo_rt_6hr, k.uo_rt_12hr, k.uo_rt_24hr
    , k.aki_stage_uo
    , ROW_NUMBER() OVER 
    (
      PARTITION BY k.icustay_id
      ORDER BY k.aki_stage_uo DESC, k.uo_rt_24hr DESC, k.uo_rt_12hr DESC, k.uo_rt_6hr DESC
    ) AS rn
  FROM icustays ie
  INNER JOIN kdigo_stages k
    ON ie.icustay_id = k.icustay_id
  WHERE k.charttime > (ie.intime - interval '6' hour)
  AND k.charttime <= (ie.intime + interval '48' hour)
  AND k.aki_stage_uo IS NOT NULL
), aki_final as (
-- final table is aki_stage, include worst cr/uo for convenience
select
    ie.icustay_id
  , cr.charttime as charttime_creat
  , cr.creat
  , cr.aki_stage_creat
  , uo.charttime as charttime_uo
  , uo.uo_rt_6hr
  , uo.uo_rt_12hr
  , uo.uo_rt_24hr
  , uo.aki_stage_uo

  -- Classify AKI using both creatinine/urine output criteria
  , GREATEST(cr.aki_stage_creat,uo.aki_stage_uo) AS aki_stage_48hr
  , CASE WHEN GREATEST(cr.aki_stage_creat, uo.aki_stage_uo) > 0 THEN 1 ELSE 0 END AS aki_48hr

FROM icustays ie
LEFT JOIN cr_aki cr
  ON ie.icustay_id = cr.icustay_id
  AND cr.rn = 1
LEFT JOIN uo_aki uo
  ON ie.icustay_id = uo.icustay_id
  AND uo.rn = 1
order by ie.icustay_id)
, final as (
select icud.*, icustays.FIRST_CAREUNIT, hw.weight_first, hw.weight_last, hw.height_first, hw.height_last,
case when patients.dod <= icud.outtime then 1 else 0 end as icu_mortality,
vtd.vent_duration_days, 
fluid_balance.nettotal,
cormob.hypertension, cormob.CONGESTIVE_HEART_FAILURE,
case when cormob.DIABETES_UNCOMPLICATED + cormob.DIABETES_COMPLICATED >= 1 then 1 else 0 end as diabetes,
add_cormor.* , aki_final.aki_48hr, angus_sepsis.angus as sepsis, vaspor.vaspor, vaspin.vaspin,
aps.apsiii, aps.apsiii_prob, sofa.SOFA
from icustay_detail icud
left join icustays on icud.icustay_id = icustays.icustay_id
left join sofa on icud.icustay_id = sofa.icustay_id
left join patients on icud.subject_id = patients.subject_id
left join hw on icud.icustay_id = hw.ICUSTAY_ID 
left join vtd on icud.icustay_id = vtd.icustay_id 
left join fluid_balance on icud.icustay_id = fluid_balance.icustay_id 
left join elixhauser_quan cormob on icud.hadm_id = cormob.hadm_id
left join add_cormor on icud.hadm_id = add_cormor.HADM_ID 
left join angus_sepsis angus_sepsis on angus_sepsis.hadm_id = icud.hadm_id 
left join aki_final on icud.icustay_id = aki_final.icustay_id
left join (select icustay_id, min(vasonum) as vaspor from vasopressordurations group by icustay_id) vaspor
on icud.icustay_id = vaspor.icustay_id 
left join (select icustay_id, min(vasonum) as vaspin from vasopressindurations group by icustay_id) vaspin
on icud.icustay_id = vaspin.icustay_id 
left join apsiii aps on aps.icustay_id = icud.icustay_id 
where first_hosp_stay = 'Y'
and first_icu_stay = 'Y')
select 
*
from final
"""
data = pd.read_sql_query(query,con)
data.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,los_hospital,age,ethnicity,admission_type,...,cereb,ihd,cancer,aki_48hr,sepsis,vaspor,vaspin,apsiii,apsiii_prob,sofa
0,55853,100010,271147,F,2109-12-10 07:15:00,2109-12-14 16:45:00,4.3958,54.5208,WHITE,ELECTIVE,...,0,0,1,1,0,1.0,NaN,26,0.038894,2
1,87977,100011,214619,M,2177-08-29 04:51:00,2177-09-12 14:24:00,14.3979,21.5041,HISPANIC OR LATINO,EMERGENCY,...,0,0,0,0,1,1.0,NaN,37,0.063722,2
2,1189,100023,281211,M,2130-05-28 05:59:00,2130-05-30 14:38:00,2.3604,0.0007,WHITE,NEWBORN,...,0,0,0,0,0,NaN,NaN,10,0.018644,0
3,53456,100028,297261,F,2142-12-23 18:06:00,2142-12-30 15:45:00,6.9021,72.5238,WHITE,EMERGENCY,...,0,0,0,0,1,NaN,NaN,53,0.126616,5
4,45631,100040,277779,M,2193-06-27 18:52:00,2193-06-30 16:00:00,2.8806,30.5545,BLACK/AFRICAN AMERICAN,EMERGENCY,...,0,0,0,0,0,NaN,NaN,27,0.040700,1


In [133]:
data.to_csv('table1.csv', index=False)

### Cohort

In [128]:
query = """
with base_raw as (
select icud.*, ct.CHARTTIME, ct.itemid, ct.VALUE,
(case when itemid in (51,6701,220050) and valuenum > 0 and valuenum < 300 then 'SBP'  
      when itemid in (8368,8555,220051) and valuenum > 0 and valuenum < 200 then 'DBP'
      when itemid in (52,6702,220052,225312) and valuenum > 0 and valuenum < 250 then 'MBP' else null end) as BPtype
from icustay_detail as icud
join chartevents ct
on icud.icustay_id = ct.ICUSTAY_ID
where first_hosp_stay = 'Y'
and first_icu_stay = 'Y'
and itemid in 
(  51, --	Arterial BP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220051, --	Arterial Blood Pressure diastolic

  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  220052, --"Arterial Blood Pressure mean"
  225312 --"ART BP mean"
  )
),
BP_time as (
select *, 
extract(epoch from (charttime - intime)) / 60.0 as current_offset, 
extract(epoch from (first_value(charttime) over (partition by icustay_id order by charttime) - intime)) / 60.0 as first_offset,
extract(epoch from (first_value(charttime) over (partition by icustay_id order by charttime desc) -
first_value(charttime) over (partition by icustay_id order by charttime asc))) / 60.0 as first_last_interval
from base_raw
where BPtype is not null
),
agg_bp as (
select icustay_id,
sum(case when BPtype = 'MBP' then 1 else 0 end) as MBPcounts,
sum(case when BPtype = 'SBP' then 1 else 0 end) as SBPcounts,
sum(case when BPtype = 'DBP' then 1 else 0 end) as DBPcounts
from BP_time
where first_offset <= 24 * 60 -- criteria 1
-- and current_offset <= 24 * 60
and first_last_interval >= 24 * 60 -- criteria 2
group by icustay_id
)

select * from agg_bp
where MBPcounts >= 12 or (SBPcounts >= 12 and DBPcounts >= 12) -- criteria 3

"""
data = pd.read_sql_query(query,con)
data.head()

,icustay_id,mbpcounts,sbpcounts,dbpcounts
0,293745,73,74,74
1,282669,146,117,117
2,271214,87,92,92
3,289494,464,455,455
4,205331,316,318,318


In [129]:
len(data)

13346

In [107]:
data.to_csv('cohort.csv', index=False)

### Table 2

In [108]:
query = """
with ce as
(
  select ce.icustay_id
    , ce.charttime
    , (case when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then valuenum else null end) as HeartRate
    , (case when itemid in (51,6701,220050) and valuenum > 0 and valuenum < 400 then valuenum else null end) as SysBP
    , (case when itemid in (8368,8555,220051) and valuenum > 0 and valuenum < 300 then valuenum else null end) as DiasBP
    , (case when itemid in (52,6702,220052,225312) and valuenum > 0 and valuenum < 300 then valuenum else null end) as MeanBP
    , (case when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then valuenum else null end) as RespRate
    , (case when itemid in (223761,678) and valuenum > 70 and valuenum < 120 then (valuenum-32)/1.8 -- converted to degC in valuenum call
               when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then valuenum else null end) as TempC
    , (case when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then valuenum else null end) as SpO2
    , (case when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then valuenum else null end) as Glucose
  from chartevents ce
  -- exclude rows marked as error
  where (ce.error IS null or ce.error = 0)
  and ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --	Arterial BP [Systolic]
  6701, --	Arterial BP #2 [Systolic]
  220050, --	Arterial Blood Pressure systolic

  8368, --	Arterial BP [Diastolic]
  8555, --	Arterial BP #2 [Diastolic]
  220051, --	Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  52, --"Arterial BP Mean"
  6702, --	Arterial BP Mean #2
  220052, --"Arterial Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--	Respiratory Rate
  615,--	Resp Rate (Total)
  220210,--	Respiratory Rate
  224690, --	Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab and fingerstick
  807,--	Fingerstick Glucose
  811,--	Glucose (70-105)
  1529,--	Glucose
  3745,--	BloodGlucose
  3744,--	Blood Glucose
  225664,--	Glucose finger stick
  220621,--	Glucose (serum)
  226537,--	Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,	-- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --	"Temperature F"

  )
), pivoted_vital as (
select
    ce.icustay_id
  , ce.charttime
  , avg(HeartRate) as HeartRate
  , avg(SysBP) as SysBP
  , avg(DiasBP) as DiasBP
  , avg(MeanBP) as MeanBP
  , avg(RespRate) as RespRate
  , avg(TempC) as TempC
  , avg(SpO2) as SpO2
  , avg(Glucose) as Glucose
from ce
group by ce.icustay_id, ce.charttime
order by ce.icustay_id, ce.charttime)
, raw as (
select pv.*, icud.intime, 
extract(epoch from (charttime - icud.intime)) / 60.0 as chartoffset
from pivoted_vital pv
join icustay_detail icud
on pv.icustay_id = icud.icustay_id 
where icud.first_hosp_stay = 'Y'
and icud.first_icu_stay = 'Y')
select * from raw
order by icustay_id asc, charttime asc

"""
data = pd.read_sql_query(query,con)
data.head()

,icustay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,glucose,intime,chartoffset
0,200003,2199-08-02 19:15:00,132.0,NaN,NaN,NaN,25.0,39.555554,97.0,NaN,2199-08-02 19:50:04,-35.066667
1,200003,2199-08-02 20:00:00,119.0,NaN,NaN,NaN,35.0,NaN,97.0,NaN,2199-08-02 19:50:04,9.933333
2,200003,2199-08-02 21:00:00,122.0,NaN,NaN,NaN,34.0,38.999998,97.0,NaN,2199-08-02 19:50:04,69.933333
3,200003,2199-08-02 21:25:00,115.0,NaN,NaN,NaN,33.0,NaN,96.0,NaN,2199-08-02 19:50:04,94.933333
4,200003,2199-08-02 21:30:00,116.0,NaN,NaN,NaN,34.0,NaN,95.0,NaN,2199-08-02 19:50:04,99.933333


In [109]:
data.to_csv('table2.csv', index=False)

In [121]:
query = """
with cr as
(
select
    ie.icustay_id
  , ie.intime, ie.outtime
  , le.valuenum as creat
  , le.charttime
  from icustays ie
  left join labevents le
    on ie.subject_id = le.subject_id
    and le.ITEMID = 50912
    and le.VALUENUM is not null
    and le.CHARTTIME between (ie.intime - interval '7' day) and (ie.intime + interval '3' day)
)
, kdigo_creat as (
-- add in the lowest value in the previous 48 hours/7 days
SELECT
  cr.icustay_id
  , cr.charttime
  , cr.creat
  , MIN(cr24.creat) AS creat_low_past_24hr
  , MIN(cr48.creat) AS creat_low_past_48hr
  , MIN(cr72.creat) AS creat_low_past_72hr
  , MIN(cr72.creat) AS creat_low_past_7day
FROM cr
LEFT JOIN cr cr24
  ON cr.icustay_id = cr24.icustay_id
  AND cr24.charttime <  cr.charttime
  AND cr24.charttime >= (cr.charttime - INTERVAL '24' HOUR)
LEFT JOIN cr cr48
  ON cr.icustay_id = cr48.icustay_id
  AND cr48.charttime <  cr.charttime
  AND cr48.charttime >= (cr.charttime - INTERVAL '48' HOUR)
LEFT JOIN cr cr72
  ON cr.icustay_id = cr72.icustay_id
  AND cr72.charttime <  cr.charttime
  AND cr72.charttime >= (cr.charttime - INTERVAL '72' HOUR)
LEFT JOIN cr cr7d
  ON cr.icustay_id = cr7d.icustay_id
  AND cr7d.charttime <  cr.charttime
  AND cr7d.charttime >= (cr.charttime - INTERVAL '7' DAY)
GROUP BY cr.icustay_id, cr.charttime, cr.creat
ORDER BY cr.icustay_id, cr.charttime, cr.creat)

-----
, ur_stg as
(
  select io.icustay_id, io.charttime
  -- we have joined each row to all rows preceding within 24 hours
  -- we can now sum these rows to get total UO over the last 24 hours
  -- we can use case statements to restrict it to only the last 6/12 hours
  -- therefore we have three sums:
  -- 1) over a 6 hour period
  -- 2) over a 12 hour period
  -- 3) over a 24 hour period
  -- note that we assume data charted at charttime corresponds to 1 hour of UO
  -- therefore we use '5' and '11' to restrict the period, rather than 6/12
  -- this assumption may overestimate UO rate when documentation is done less than hourly

  -- 6 hours
  , sum(case when io.charttime <= iosum.charttime + interval '5' hour
      then iosum.VALUE
    else null end) as UrineOutput_6hr
  -- 12 hours
  , sum(case when io.charttime <= iosum.charttime + interval '11' hour
      then iosum.VALUE
    else null end) as UrineOutput_12hr
  -- 24 hours
  , sum(iosum.VALUE) as UrineOutput_24hr
  -- calculate the number of hours over which we've tabulated UO
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - 
        -- below MIN() gets the earliest time that was used in the summation 
        MIN(case when io.charttime <= iosum.charttime + interval '5' hour
          then iosum.charttime
        else null end)
    -- convert from EPOCH (seconds) to hours by dividing by 360.0
    )/3600.0 AS NUMERIC), 4) AS uo_tm_6hr
  -- repeat extraction for 12 hours and 24 hours
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - 
        MIN(case when io.charttime <= iosum.charttime + interval '11' hour
          then iosum.charttime
        else null end)
   )/3600.0 AS NUMERIC), 4) AS uo_tm_12hr
  , ROUND(CAST(EXTRACT(EPOCH FROM
      io.charttime - MIN(iosum.charttime)
   )/3600.0 AS NUMERIC), 4) AS uo_tm_24hr
  from urineoutput io
  -- this join gives all UO measurements over the 24 hours preceding this row
  left join urineoutput iosum
    on  io.icustay_id = iosum.icustay_id
    and io.charttime >= iosum.charttime
    and io.charttime <= (iosum.charttime + interval '23' hour)
  group by io.icustay_id, io.charttime
)
, kdigo_uo as (
select
  ur.icustay_id
, ur.charttime
, wd.weight
, ur.UrineOutput_6hr
, ur.UrineOutput_12hr
, ur.UrineOutput_24hr
-- calculate rates - adding 1 hour as we assume data charted at 10:00 corresponds to previous hour
, ROUND((ur.UrineOutput_6hr/wd.weight/(uo_tm_6hr+1))::NUMERIC, 4) AS uo_rt_6hr
, ROUND((ur.UrineOutput_12hr/wd.weight/(uo_tm_12hr+1))::NUMERIC, 4) AS uo_rt_12hr
, ROUND((ur.UrineOutput_24hr/wd.weight/(uo_tm_24hr+1))::NUMERIC, 4) AS uo_rt_24hr
-- time of earliest UO measurement that was used to calculate the rate
, uo_tm_6hr
, uo_tm_12hr
, uo_tm_24hr
from ur_stg ur
left join weightdurations wd
  on  ur.icustay_id = wd.icustay_id
  and ur.charttime >= wd.starttime
  and ur.charttime <  wd.endtime
order by icustay_id, charttime
)

-----

, cr_stg AS
(
  SELECT
    cr.icustay_id
    , cr.charttime
    , cr.creat
    , case
        -- 3x baseline
        when cr.creat >= (cr.creat_low_past_7day*3.0) then 3
        -- *OR* cr >= 4.0 with associated increase
        when cr.creat >= 4
        -- For patients reaching Stage 3 by SCr >4.0 mg/dl
        -- require that the patient first achieve ... acute increase >= 0.3 within 48 hr
        -- *or* an increase of >= 1.5 times baseline
        and (cr.creat_low_past_48hr <= 3.7 OR cr.creat >= (1.5*cr.creat_low_past_7day))
            then 3 
        -- TODO: initiation of RRT
        when cr.creat >= (cr.creat_low_past_7day*2.0) then 2
        when cr.creat >= (cr.creat_low_past_48hr+0.3) then 1
        when cr.creat >= (cr.creat_low_past_7day*1.5) then 1
    else 0 end as aki_stage_creat
  FROM kdigo_creat cr
)
-- stages for UO / creat
, uo_stg as
(
  select
      uo.icustay_id
    , uo.charttime
    , uo.weight
    , uo.uo_rt_6hr
    , uo.uo_rt_12hr
    , uo.uo_rt_24hr
    -- AKI stages according to urine output
    , CASE
        WHEN uo.uo_rt_6hr IS NULL THEN NULL
        -- require patient to be in ICU for at least 6 hours to stage UO
        WHEN uo.charttime <= ie.intime + interval '6' hour THEN 0
        -- require the UO rate to be calculated over half the period
        -- i.e. for uo rate over 24 hours, require documentation at least 12 hr apart
        WHEN uo.uo_tm_24hr >= 11 AND uo.uo_rt_24hr < 0.3 THEN 3
        WHEN uo.uo_tm_12hr >= 5 AND uo.uo_rt_12hr = 0 THEN 3
        WHEN uo.uo_tm_12hr >= 5 AND uo.uo_rt_12hr < 0.5 THEN 2
        WHEN uo.uo_tm_6hr >= 2 AND uo.uo_rt_6hr  < 0.5 THEN 1
    ELSE 0 END AS aki_stage_uo
  from kdigo_uo uo
  INNER JOIN icustays ie
    ON uo.icustay_id = ie.icustay_id
)
-- get all charttimes documented
, tm_stg AS
(
    SELECT
      icustay_id, charttime
    FROM cr_stg
    UNION
    SELECT
      icustay_id, charttime
    FROM uo_stg
)
, kdigo_stages as (select
    ie.icustay_id
  , tm.charttime
  , cr.creat
  , cr.aki_stage_creat
  , uo.uo_rt_6hr
  , uo.uo_rt_12hr
  , uo.uo_rt_24hr
  , uo.aki_stage_uo
  -- Classify AKI using both creatinine/urine output criteria
  , GREATEST(cr.aki_stage_creat, uo.aki_stage_uo) AS aki_stage
FROM icustays ie
-- get all possible charttimes as listed in tm_stg
LEFT JOIN tm_stg tm
  ON ie.icustay_id = tm.icustay_id
LEFT JOIN cr_stg cr
  ON ie.icustay_id = cr.icustay_id
  AND tm.charttime = cr.charttime
LEFT JOIN uo_stg uo
  ON ie.icustay_id = uo.icustay_id
  AND tm.charttime = uo.charttime
order by ie.icustay_id, tm.charttime)

, cr_aki AS
(
  SELECT
    k.icustay_id
    , k.charttime
    , k.creat
    , k.aki_stage_creat
    , ROW_NUMBER() OVER (PARTITION BY k.icustay_id ORDER BY k.aki_stage_creat DESC, k.creat DESC) AS rn
  FROM icustays ie
  INNER JOIN kdigo_stages k
    ON ie.icustay_id = k.icustay_id
  WHERE k.charttime > (ie.intime - interval '6' hour)
  AND k.charttime <= (ie.intime + interval '48' hour)
  AND k.aki_stage_creat IS NOT NULL
)
-- get the worst staging of urine output in the first 48 hours
, uo_aki AS
(
  SELECT
    k.icustay_id
    , k.charttime
    , k.uo_rt_6hr, k.uo_rt_12hr, k.uo_rt_24hr
    , k.aki_stage_uo
    , ROW_NUMBER() OVER 
    (
      PARTITION BY k.icustay_id
      ORDER BY k.aki_stage_uo DESC, k.uo_rt_24hr DESC, k.uo_rt_12hr DESC, k.uo_rt_6hr DESC
    ) AS rn
  FROM icustays ie
  INNER JOIN kdigo_stages k
    ON ie.icustay_id = k.icustay_id
  WHERE k.charttime > (ie.intime - interval '6' hour)
  AND k.charttime <= (ie.intime + interval '48' hour)
  AND k.aki_stage_uo IS NOT NULL
), aki_final as (
-- final table is aki_stage, include worst cr/uo for convenience
select
    ie.icustay_id
  , cr.charttime as charttime_creat
  , cr.creat
  , cr.aki_stage_creat
  , uo.charttime as charttime_uo
  , uo.uo_rt_6hr
  , uo.uo_rt_12hr
  , uo.uo_rt_24hr
  , uo.aki_stage_uo

  -- Classify AKI using both creatinine/urine output criteria
  , GREATEST(cr.aki_stage_creat,uo.aki_stage_uo) AS aki_stage_48hr
  , CASE WHEN GREATEST(cr.aki_stage_creat, uo.aki_stage_uo) > 0 THEN 1 ELSE 0 END AS aki_48hr

FROM icustays ie
LEFT JOIN cr_aki cr
  ON ie.icustay_id = cr.icustay_id
  AND cr.rn = 1
LEFT JOIN uo_aki uo
  ON ie.icustay_id = uo.icustay_id
  AND uo.rn = 1
order by ie.icustay_id)
select * from aki_final limit 100
"""
data = pd.read_sql_query(query,con)
data.head()

,icustay_id,charttime_creat,creat,aki_stage_creat,charttime_uo,uo_rt_6hr,uo_rt_12hr,uo_rt_24hr,aki_stage_uo,aki_stage_48hr,aki_48hr
0,200001,2181-11-27 05:54:00,3.5,1.0,NaT,NaN,NaN,NaN,NaN,1.0,1
1,200003,2199-08-03 01:47:00,1.0,0.0,2199-08-02 20:15:00,2.9677,2.9677,2.9677,0.0,0.0,0
2,200006,2159-09-03 08:00:00,1.1,0.0,2159-09-04 06:00:00,0.6553,0.4799,1.2386,2.0,2.0,1
3,200007,2109-02-17 08:44:00,0.9,0.0,2109-02-17 16:00:00,1.5873,1.5873,1.5873,0.0,0.0,0
4,200009,2189-12-02 03:25:00,0.5,0.0,2189-12-02 06:00:00,0.4434,0.4886,0.5813,2.0,2.0,1
